In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [2]:
dataset = pd.read_csv("obesity.csv")
print("Classes in the dataset:", dataset['NObeyesdad'].unique())

Classes in the dataset: ['Normal_Weight' 'Overweight_Level_I' 'Overweight_Level_II'
 'Obesity_Type_I' 'Insufficient_Weight' 'Obesity_Type_II'
 'Obesity_Type_III']


One hot encoding

In [3]:
categories = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'CAEC', 'CALC', 'MTRANS', 'NObeyesdad']
for c in categories:
  dummies = pd.get_dummies(dataset[c], prefix=(str(c)+"_"))
  dataset = pd.concat([dataset, dummies], axis=1)
dataset = dataset.drop(columns=categories)

In [4]:
d = ['NObeyesdad__Insufficient_Weight',
       'NObeyesdad__Normal_Weight', 'NObeyesdad__Obesity_Type_I',
       'NObeyesdad__Obesity_Type_II', 'NObeyesdad__Obesity_Type_III',
       'NObeyesdad__Overweight_Level_I', 'NObeyesdad__Overweight_Level_II']
X = dataset.drop(columns=d)
y = dataset[d]
print(y.columns)

Index(['NObeyesdad__Insufficient_Weight', 'NObeyesdad__Normal_Weight',
       'NObeyesdad__Obesity_Type_I', 'NObeyesdad__Obesity_Type_II',
       'NObeyesdad__Obesity_Type_III', 'NObeyesdad__Overweight_Level_I',
       'NObeyesdad__Overweight_Level_II'],
      dtype='object')


In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_rescaled = scaler.fit_transform(X)
X = pd.DataFrame(data = X_rescaled, columns = X.columns)

Split the data and run the model

In [7]:
data_train, data_test, class_train, class_test = train_test_split(X, y, test_size=0.2)
print(data_train.shape)
print(class_train.shape)
mlp = MLPClassifier(solver = 'sgd', activation = 'logistic', learning_rate_init = 0.4, batch_size = 100, hidden_layer_sizes = (17, 20), max_iter = 600)

(1688, 31)
(1688, 7)


In [8]:
mlp.fit(data_train, class_train)

MLPClassifier(activation='logistic', batch_size=100,
              hidden_layer_sizes=(17, 20), learning_rate_init=0.4, max_iter=600,
              solver='sgd')

In [9]:
pred = mlp.predict(data_test)

In [10]:
print(pred)

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]]


In [11]:
print("Accuracy : ", accuracy_score(class_test, pred))
print("Mean Square Error : ", mean_squared_error(class_test, pred))

Accuracy :  0.9314420803782506
Mean Square Error :  0.018574805808848367


In [12]:
print(multilabel_confusion_matrix(class_test, pred))

print("Classification Report : ")
print(classification_report(class_test, pred))

[[[361   2]
  [  5  55]]

 [[359   9]
  [  6  49]]

 [[347   1]
  [  4  71]]

 [[358   0]
  [  1  64]]

 [[364   0]
  [  0  59]]

 [[369   8]
  [  7  39]]

 [[352   8]
  [  4  59]]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        60
           1       0.84      0.89      0.87        55
           2       0.99      0.95      0.97        75
           3       1.00      0.98      0.99        65
           4       1.00      1.00      1.00        59
           5       0.83      0.85      0.84        46
           6       0.88      0.94      0.91        63

   micro avg       0.93      0.94      0.94       423
   macro avg       0.93      0.93      0.93       423
weighted avg       0.94      0.94      0.94       423
 samples avg       0.93      0.94      0.93       423



Cross Validation

In [13]:
from sklearn.model_selection import cross_validate
mlpCV = cross_validate(mlp, X, y, cv=10, scoring=['accuracy', 'neg_mean_squared_error'])
#print all the accuracy values from each iteration
print('Accuracy')
print(mlpCV['test_accuracy'])
#print all the MSE values from each iteration
print('MSE')
print(-1*mlpCV['test_neg_mean_squared_error'])
print('Average Accuracy = ',  sum(mlpCV['test_accuracy'])/len(mlpCV['test_accuracy']))
print('Average MSE = ', sum(-1 * mlpCV['test_neg_mean_squared_error']) / len(mlpCV['test_neg_mean_squared_error']))

Accuracy
[0.72169811 0.8056872  0.93364929 0.96208531 0.89099526 0.84834123
 0.97156398 0.99052133 0.99052133 1.        ]
MSE
[0.07614555 0.0534868  0.01760325 0.00947867 0.02911307 0.03926879
 0.00541638 0.0013541  0.00203114 0.        ]
Average Accuracy =  0.9115063042117499
Average MSE =  0.023389775296687575


Hyperparameter Tuning

In [14]:
#MLP hyperparameter tuning

#logic from hw3demo
#set up parameters
max_iterations = [500,600,700]
hidden_layer_size = [(17, 20), (17,20,15), (17, 20, 20)]
activations = ["logistic", "relu", "tanh"]
learning_rate_inits = [0.3,0.4,0.5]
params = dict(activation = activations, hidden_layer_sizes = hidden_layer_size, max_iter = max_iterations, learning_rate_init = learning_rate_inits)
grid = GridSearchCV(estimator = mlp, param_grid=params, scoring="accuracy")
grid.fit(X, y)

GridSearchCV(estimator=MLPClassifier(activation='logistic', batch_size=100,
                                     hidden_layer_sizes=(17, 20),
                                     learning_rate_init=0.4, max_iter=600,
                                     solver='sgd'),
             param_grid={'activation': ['logistic', 'relu', 'tanh'],
                         'hidden_layer_sizes': [(17, 20), (17, 20, 15),
                                                (17, 20, 20)],
                         'learning_rate_init': [0.3, 0.4, 0.5],
                         'max_iter': [500, 600, 700]},
             scoring='accuracy')

In [15]:
#results
print("Optimal Hyper-Parameters:", grid.best_params_)
print("Optimal Accuracy:", grid.best_score_)

Optimal Hyper-Parameters: {'activation': 'logistic', 'hidden_layer_sizes': (17, 20), 'learning_rate_init': 0.4, 'max_iter': 700}
Optimal Accuracy: 0.8309995182234771


In [19]:
#make optimal MLP
optimalMLP = MLPClassifier(solver = 'sgd', random_state = 42, activation = 'logistic', learning_rate_init = 0.4, batch_size = 100, hidden_layer_sizes = (17, 20), max_iter = 700)
#fit and predict
optimalMLP.fit(data_train, class_train)
pred = optimalMLP.predict(data_test)
#results
print("Accuracy : ", accuracy_score(class_test, pred))
print("Mean Square Error : ", mean_squared_error(class_test, pred))

Accuracy :  0.9408983451536643
Mean Square Error :  0.014859844647078692


In [20]:
#MORE results
print(multilabel_confusion_matrix(class_test, pred))

print("Classification Report : ")
print(classification_report(class_test, pred))

[[[360   3]
  [  2  58]]

 [[367   1]
  [ 10  45]]

 [[348   0]
  [  1  74]]

 [[358   0]
  [  0  65]]

 [[364   0]
  [  0  59]]

 [[365  12]
  [  5  41]]

 [[357   3]
  [  7  56]]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.95      0.97      0.96        60
           1       0.98      0.82      0.89        55
           2       1.00      0.99      0.99        75
           3       1.00      1.00      1.00        65
           4       1.00      1.00      1.00        59
           5       0.77      0.89      0.83        46
           6       0.95      0.89      0.92        63

   micro avg       0.95      0.94      0.95       423
   macro avg       0.95      0.94      0.94       423
weighted avg       0.96      0.94      0.95       423
 samples avg       0.94      0.94      0.94       423



In [21]:
#Cross Validation for Optimized MLP
optimal_CV = cross_validate(optimalMLP, X, y, cv=10, scoring=['accuracy', 'neg_mean_squared_error'])
#print all the accuracy values from each iteration
print('Accuracy')
print(optimal_CV['test_accuracy'])
#print all the MSE values from each iteration
print('MSE')
print(-1*optimal_CV['test_neg_mean_squared_error'])

print('Average Accuracy = ', sum(optimal_CV['test_accuracy']) / len(optimal_CV['test_accuracy']))
print('Average MSE = ', sum(-1 * optimal_CV['test_neg_mean_squared_error']) / len(optimal_CV['test_neg_mean_squared_error']))

Accuracy
[0.66037736 0.72985782 0.94312796 0.97630332 0.8436019  0.99052133
 0.96682464 0.99052133 1.         1.        ]
MSE
[0.0902965  0.07109005 0.01557211 0.00473934 0.03452945 0.00203114
 0.00880162 0.00203114 0.         0.        ]
Average Accuracy =  0.9101135652329428
Average MSE =  0.022909135039153816
